# 数据准备

主要用torchtext工具包

## 导入包

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchtext.legacy import data
from torchtext.legacy import datasets
import torch.optim as optim
import time
import random

SEED = 1234

torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

## 下载数据集

下载IMDb数据集，分成train/test两个torchtext.datasets类别。

数据用Fields处理：
- TorchText中的Field决定你的数据会被怎样处理。
- 情感分类中，所需要接触到的数据由文本字符串和两种情感（pos/neg）
- Field的参数制定了数据会被如何处理。
  - 使用TEXT field定义如何处理电影评论
  - 使用LABEL field处理两个情感类别

In [2]:
TEXT = data.Field(tokenize='spacy')
LABEL = data.LabelField(dtype=torch.float)

C:\Users\Huris\anaconda3\envs\sEMG\lib\site-packages\torchtext\data\utils.py:123: UserWarning: Spacy model "en" could not be loaded, trying "en_core_web_sm" instead
  warnings.warn(f'Spacy model "{language}" could not be loaded, trying "{OLD_MODEL_SHORTCUTS[language]}" instead')


注意这里要安装一个spacy包，使用命令：
- pip install spacy
- python -m spacy download en

如果第二条命令无法运行，报网络连接错误，可以参考下面的博客：
- https://www.cnblogs.com/xiaolan-Lin/p/13286885.html

注意博客中下载的en_core_web_sm和en_core_web_md版本需要与spacy版本一致：
- https://spacy.io/models/en#en_core_web_sm

TEXT fielf带有：tokenize='spacy'，表示用spaCy tokenizer来tokenize英文句子。

如果不特别声明tokenize这个参数，则默认的分词方法是使用空格。

In [3]:
train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)

print(f'Number of training examples: {len(train_data)}')
print(f'Number of testing examples: {len(test_data)}')

Number of training examples: 25000
Number of testing examples: 25000


IMDb数据集一共有50000条电影评论，每个评论都被标注为正面或负面。

In [4]:
vars(train_data.examples[0])

{'text': ['Bromwell',
  'High',
  'is',
  'a',
  'cartoon',
  'comedy',
  '.',
  'It',
  'ran',
  'at',
  'the',
  'same',
  'time',
  'as',
  'some',
  'other',
  'programs',
  'about',
  'school',
  'life',
  ',',
  'such',
  'as',
  '"',
  'Teachers',
  '"',
  '.',
  'My',
  '35',
  'years',
  'in',
  'the',
  'teaching',
  'profession',
  'lead',
  'me',
  'to',
  'believe',
  'that',
  'Bromwell',
  'High',
  "'s",
  'satire',
  'is',
  'much',
  'closer',
  'to',
  'reality',
  'than',
  'is',
  '"',
  'Teachers',
  '"',
  '.',
  'The',
  'scramble',
  'to',
  'survive',
  'financially',
  ',',
  'the',
  'insightful',
  'students',
  'who',
  'can',
  'see',
  'right',
  'through',
  'their',
  'pathetic',
  'teachers',
  "'",
  'pomp',
  ',',
  'the',
  'pettiness',
  'of',
  'the',
  'whole',
  'situation',
  ',',
  'all',
  'remind',
  'me',
  'of',
  'the',
  'schools',
  'I',
  'knew',
  'and',
  'their',
  'students',
  '.',
  'When',
  'I',
  'saw',
  'the',
  'episode',


## 划分数据集

由于现在只有train/test两个分类，需要创建一个valid集，使用.split()创建新的分类。

默认的数据分割是70|30，如果声明split_ratio，可以改变split之间的比例。（split_ratio=0.8表示80%是训练集，20%是验证集）

如果声明random_state参数，可以确保每次分割的数据集都是一样的

In [5]:
train_data, valid_data = train_data.split(random_state=random.seed(SEED))

print(f'Number of training examples: {len(train_data)}')
print(f'Number of validation examples: {len(valid_data)}')
print(f'Number of testing examples: {len(test_data)}')

Number of training examples: 17500
Number of validation examples: 7500
Number of testing examples: 25000


## 创建词典

vocabulary把每个单词——映射到一个数字。

使用最常见的25k个单词来构建单词表，用max_size参数。

其他所有单词都用\<unk\>表示。
    
这里采用一种预训练，用的都是预训练好的glove向量。

In [6]:
TEXT.build_vocab(train_data, max_size=25000, vectors="glove.6B.100d", unk_init=torch.Tensor.normal_)
LABEL.build_vocab(train_data)

print(f"Unique tokens in TEXT vocabulary: {len(TEXT.vocab)}")  # 25002
print(f"Unique tokens in LABEL vocabulary: {len(LABEL.vocab)}")  # 2

Unique tokens in TEXT vocabulary: 25002
Unique tokens in LABEL vocabulary: 2


创建完词典，直接用stoi(string to int)或者itos(int to string)查看单词表

In [7]:
print(TEXT.vocab.itos[:10])  # ['<unk>', '<pad>', 'the', ',', '.', 'and', 'a', 'of', 'to', 'is']
print(LABEL.vocab.stoi)  # defaultdict(None, {'neg': 0, 'pos': 1})

['<unk>', '<pad>', 'the', ',', '.', 'and', 'a', 'of', 'to', 'is']
defaultdict(None, {'neg': 0, 'pos': 1})


把句子传进模型时，按照一个个batch传进去，即一次传入若干个句子，并且每个batch中的句子必须是相同的长度。

为了确保句子长度相同，TorchText会把短的句子pad到和最长的句子等长。

## 创建iterators

每个iteration都会返回一个batch的examples，使用BucketIterator。

BucketIterator会把长度差不多的句子放到同一个batch中，确保每个batch中不出现太多的padding。

此处模型代码有一个问题，就是把\<pad\>也当作模型的输入进行训练，更好地做法是在模型中把\<pad\>产生的输出给清除掉。

如果有GPU，还可以指定每个iteration返回的tensor都在GPU上。

In [8]:
BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Iterator的建立
train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), batch_size=BATCH_SIZE,device=device)

# 看一个样本长啥样，Iterator里，每个单词都用词典中的位置代替，每一列是一个样本
[TEXT.vocab.itos[i] for i in next(iter(train_iterator)).text[1, :]]

['acting',
 'is',
 'luck',
 ':',
 'encouraged',
 'makes',
 'is',
 'Cause',
 'I',
 "'s",
 'have',
 'quite',
 'stunning',
 'Dix',
 'find',
 'br',
 'finished',
 'is',
 'and',
 'Lugosi',
 'this',
 'watched',
 'fans',
 'movie',
 'started',
 'best',
 'really',
 'find',
 'Almighty',
 'too',
 'friend',
 'is',
 'watching',
 'is',
 'suspenseful',
 'Girlfight',
 'show',
 'do',
 'times',
 'Michael',
 'had',
 'director',
 'off',
 'went',
 'was',
 'fans',
 ',',
 '<unk>',
 'the',
 'gently',
 'I',
 'I',
 'rented',
 'author',
 'wonderful',
 'this',
 'is',
 'love',
 'enjoy',
 'on',
 'movie',
 'all',
 'absolutely',
 'day']

# 模型建立与训练

## Word Averaging模型

Word Averaging模型把每个单词都通过Embedding层投射成word embedding vector，然后把一句话中的所有word vector做一个平均，就是整个句子的vector表示，之后把这个句子vector传入一个Linear层，做分类即可。
<img style="float: center;" src="images/9.png" width="70%">

网络的前向传播是这样的：
- 首先输入text，即上面准备好的数据的其中一个Iterator，大小是[seq_len, batch_size]
- 接下去通过一个embedding层，得到每个单词的embedding向量，矩阵大小[seq_len, batch_size, embed_dim]
- 把前两个维度换一下，[batch_size, seq_len, embed_dim]，接下来就是在句子的维度进行求平均，即把句子长度那个维度压扁，此处采用的是avg_pool2d来做平均池化，avg_pool2d的kernel size是(embedded.shape[1], 1)，所以句子长度的那个维度会被压扁。
- 经过这个操作后，矩阵的维度变成[batch_size, 1, embed_dim]，就是seq_len的维度上进行一个求平均，这样就得到了一句话的embedding。
- 之后通过全连接层得到最后的输出结果。

In [13]:
class WordAVGModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, output_dim, pad_idx):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=pad_idx)
        self.fc = nn.Linear(embedding_dim, output_dim)
        
    def forward(self, text):
        embedded = self.embedding(text) # [sent len, batch size, emb dim]
        embedded = embedded.permute(1, 0, 2) # [batch size, sent len, emb dim]
        pooled = F.avg_pool2d(embedded, (embedded.shape[1], 1)).squeeze(1) # [batch size, embedding_dim]
        # 这个就相当于在seq_len维度上做了一个平均
        return self.fc(pooled)

**PyTorch搭建神经网络技巧：**

先搭好大框架：
```python
class WordAVGModel(nn.Module):
	def __init__(self, ):
		pass
	def forward(self, text):
		pass
	def .....
```

之后先写forward函数，这是网络的计算过程，然后再写初始化的部分，因为可能一上来不知道网络里面到底应该有哪些网络层，先写前向传播之后，就有利于知道需要定义哪些层，这时候再写init的时候就好写了，最后再定义\_\_init\_\_里的形参，考虑\_\_init\_\_的成员对象用到了哪些需要外界传递的参数。（总体而言就是自下而上地写）

搭建完网络模型之后，就可以定义一些参数，然后建立这个网络

In [14]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
OUTPUT_DIM = 1
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

model = WordAVGModel(INPUT_DIM, EMBEDDING_DIM, OUTPUT_DIM, PAD_IDX)

网络训练部分，这个任务里给出一个embedding向量地初始化技巧，**采用已经训练好的embedding向量进行一个初始化（glove）**，这相当于网络训练的时候，embedding这部分只需要简单的微调就可以，大大加快了训练。

因此这里直接把训练好的glove参数放到embedding层的w上：类似于一种迁移思想

In [15]:
pretrained_embeddings = TEXT.vocab.vectors
model.embedding.weight.data.copy_(pretrained_embeddings)

# 相当于把embedding向量的w参数初始化成了通过大量语句训练好的embedding， 
# 这样网络训练的时候向量只需微调

# 两个特殊字符初始化为0
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]
model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

开始训练，事先写好了训练和验证函数，好处是换了模型，训练的函数不用改

In [16]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    model.train()
    
    for batch in iterator:
        optimizer.zero_grad()
        predictions = model(batch.text).squeeze(1)
        loss = criterion(predictions, batch.label)
        acc = binary_accuracy(predictions, batch.label)
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)
 
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    model.eval()
    
    with torch.no_grad():
        for batch in iterator:
            predictions = model(batch.text).squeeze(1)
            loss = criterion(predictions, batch.label)
            acc = binary_accuracy(predictions, batch.label)
            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

下面再加两个辅助函数，一个计算acc，一个计算运行时间

In [17]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum()/len(correct)
    return acc
 
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

正式训练

In [18]:
optimizer = optim.Adam(model.parameters())
criterion = nn.BCEWithLogitsLoss()   # 这个是加了sigmoid的损失函数
model = model.to(device)
criterion = criterion.to(device)

N_EPOCHS = 10

best_valid_loss = float('inf')

In [22]:
for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'wordavg-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 5s
	Train Loss: 0.684 | Train Acc: 58.54%
	 Val. Loss: 0.617 |  Val. Acc: 71.31%
Epoch: 02 | Epoch Time: 0m 3s
	Train Loss: 0.639 | Train Acc: 71.67%
	 Val. Loss: 0.505 |  Val. Acc: 75.56%
Epoch: 03 | Epoch Time: 0m 3s
	Train Loss: 0.567 | Train Acc: 78.58%
	 Val. Loss: 0.456 |  Val. Acc: 79.69%
Epoch: 04 | Epoch Time: 0m 3s
	Train Loss: 0.497 | Train Acc: 83.08%
	 Val. Loss: 0.424 |  Val. Acc: 82.67%
Epoch: 05 | Epoch Time: 0m 3s
	Train Loss: 0.438 | Train Acc: 86.10%
	 Val. Loss: 0.403 |  Val. Acc: 84.79%
Epoch: 06 | Epoch Time: 0m 3s
	Train Loss: 0.389 | Train Acc: 87.81%
	 Val. Loss: 0.400 |  Val. Acc: 86.22%
Epoch: 07 | Epoch Time: 0m 3s
	Train Loss: 0.350 | Train Acc: 89.06%
	 Val. Loss: 0.408 |  Val. Acc: 87.10%
Epoch: 08 | Epoch Time: 0m 3s
	Train Loss: 0.322 | Train Acc: 90.01%
	 Val. Loss: 0.423 |  Val. Acc: 87.68%
Epoch: 09 | Epoch Time: 0m 3s
	Train Loss: 0.295 | Train Acc: 90.92%
	 Val. Loss: 0.442 |  Val. Acc: 87.94%
Epoch: 10 | Epoch Time: 0m 3

对模型进行测试

In [19]:
model.load_state_dict(torch.load('wordavg-model.pt'))
test_loss, test_acc = evaluate(model, test_iterator, criterion)
print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.433 | Test Acc: 85.50%


实际测一下效果

In [20]:
import spacy
nlp = spacy.load('en_core_web_sm')

def predict_sentiment(sentence):
    tokenized = [tok.text for tok in nlp.tokenizer(sentence)]
    indexed = [TEXT.vocab.stoi[t] for t in tokenized]
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(1)
    prediction = torch.sigmoid(model(tensor))
    return prediction.item()

predict_sentiment("This film is terrible"), predict_sentiment("This film is great")

(8.31305413395711e-10, 1.0)

# RNN Model

尝试把模型换成一个recurrent neural network（RNN）

RNN经常用于encode一个sequence：$h_t=RNN(x_t, h_{t-1})$
- 使用最后一个hidden state$h_T$来表示整个句子
- 把$h_T$通过一个线性变换$f$，然后用于预测句子的情感

使用一个2层双向LSTM网络：
<img style="float: center;" src="images/10.png" width="70%">

网络计算过程：
- 输入一批句子，大小为[seq_len, batch_size]
- 经过一个embedding层得到每个单词的embedding向量，此时维度[seq_len, batch_size, embed_dim]
- 经过一个双向LSTM，并且是2层堆叠起来的，此时网络输出[seq_len, batch_size, hidden_size\*num_directions]
- LSTM的隐藏状态h和c是[num_layers\*num_directions, batch_size, hidden_size]，此时需要拿到最后一层最后一个时间步LSTM的隐藏层状态，对他俩进行一个拼接
- 最后通过全连接层得到结果

In [21]:
class RNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, 
                 n_layers, bidirectional, dropout, pad_idx):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=pad_idx)
        self.rnn = nn.LSTM(embedding_dim, hidden_dim, num_layers=n_layers, 
                           bidirectional=bidirectional, dropout=dropout)
        self.fc = nn.Linear(hidden_dim * 2, output_dim)
        self.dropout = nn.Dropout(dropout)
  
    def forward(self, text):
        embedded = self.dropout(self.embedding(text)) #[sent len, batch size, emb dim]
        output, (hidden, cell) = self.rnn(embedded)
        #output = [sent len, batch size, hid dim * num directions]
        #hidden = [num layers * num directions, batch size, hid dim]
        #cell = [num layers * num directions, batch size, hid dim]
    
        #concat the final forward (hidden[-2,:,:]) and backward (hidden[-1,:,:]) hidden layers
        hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim=1)) # [batch size, hid dim * num directions]
        
        #and apply dropout
        return self.fc(hidden.squeeze(0))

前向传播时，标注一下每一层之后变量的维度变化。（一个好习惯）

之后是训练

In [22]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 100
OUTPUT_DIM = 1
N_LAYERS = 2
BIDIRECTIONAL = True
DROPOUT = 0.5
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

model = RNN(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM, 
            N_LAYERS, BIDIRECTIONAL, DROPOUT, PAD_IDX)
optimizer = optim.Adam(model.parameters())
criterion = nn.BCEWithLogitsLoss()   # 这个是加了sigmoid的损失函数
model = model.to(device)
criterion = criterion.to(device)

N_EPOCHS = 10

best_valid_loss = float('inf')

In [34]:
for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'lstm-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 47s
	Train Loss: 0.681 | Train Acc: 55.56%
	 Val. Loss: 0.645 |  Val. Acc: 63.93%
Epoch: 02 | Epoch Time: 0m 46s
	Train Loss: 0.641 | Train Acc: 63.97%
	 Val. Loss: 0.593 |  Val. Acc: 70.76%
Epoch: 03 | Epoch Time: 0m 45s
	Train Loss: 0.679 | Train Acc: 56.51%
	 Val. Loss: 0.650 |  Val. Acc: 61.66%
Epoch: 04 | Epoch Time: 0m 45s
	Train Loss: 0.603 | Train Acc: 67.76%
	 Val. Loss: 0.604 |  Val. Acc: 69.84%
Epoch: 05 | Epoch Time: 0m 46s
	Train Loss: 0.565 | Train Acc: 71.22%
	 Val. Loss: 0.543 |  Val. Acc: 76.31%
Epoch: 06 | Epoch Time: 0m 46s
	Train Loss: 0.498 | Train Acc: 76.70%
	 Val. Loss: 0.501 |  Val. Acc: 77.04%
Epoch: 07 | Epoch Time: 0m 46s
	Train Loss: 0.448 | Train Acc: 79.46%
	 Val. Loss: 0.414 |  Val. Acc: 81.41%
Epoch: 08 | Epoch Time: 0m 45s
	Train Loss: 0.438 | Train Acc: 80.36%
	 Val. Loss: 0.468 |  Val. Acc: 78.95%
Epoch: 09 | Epoch Time: 0m 45s
	Train Loss: 0.507 | Train Acc: 76.17%
	 Val. Loss: 0.471 |  Val. Acc: 77.41%
Epoch: 10 | Epoch T

In [23]:
model.load_state_dict(torch.load('lstm-model.pt'))
test_loss, test_acc = evaluate(model, test_iterator, criterion)
print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.432 | Test Acc: 80.75%


# CNN Model

CNN也可以处理语言序列。

卷积神经网络的计算过程相比于前面两个网络模型有点难理解，因此CNN一般用于处理图像，而语言模型是时序数据，CNN如何捕捉时序序列之间的关联呢？

此处利用卷积+池化的操作，不过这次用的卷积核不是正方形的，其网络结构来自于：
- 《Convolutional Neural Networks for Setence Classification》
<img style="float: center;" src="images/11.png" width="70%">

前向传播的过程：
- 首先接收一批句子，维度是[batch_size, seq_len]，注意这里先转换了维度
- 之后经过embedding层，得到[batch_size, seq_len, emb_dim]
- 2维卷积接收的输入是4维的，因为要把这个东西看成一个图像才能进行二维卷积
  - 2维卷积层接收的输入是[batch_size, in_chnnels, Height, Width]
  - 而embedding之后是3维的，因此需要在第2个维度扩扩出一个1来（表示通道数是1维）
- 后面的seq_len和emb_dim表示图像的长和宽，之后在这里做卷积
- 卷积核大小[filter_size, emb_dim]，这样对[seq_len, emb_dim]进行卷积后，就会把emb_dim这个维度变成1，从而提取了每个单词之间的特征
- 这时候输出维度是[batch_size, num_filters, seq_len-filter_size+1, 1]
  - 这次卷积就是在embedding向量之后的后2维上进行卷积提取单词之间的特征
  - 无非就是卷积核的第2个维度是emb_dim，卷积的时候把每个单词全部的词向量都进行了运算，因为考虑了全部的词向量后才是一个完整的单词
  - 第二个维度是过滤器的个数。
- 得到了上面这个向量，把最后一个维度去掉，然后在seq_len-filter_size+1这个维度上进行最大池化，这个表示新的单词与单词间的那种关系，得到一个[batch_size, num_filters, 1]的张量，然后把第二维度去掉，再进行全连接层。

In [24]:
class CNN(nn.Module):
    def __init__(self, vocab_size, embedding_size, num_filters, filter_size, out_size, dropout, pad_idx):
        super(CNN, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_size, padding_idx=pad_idx)
        self.conv = nn.Conv2d(in_channels=1, out_channels=num_filters, kernel_size=(filter_size, embedding_size))
        self.linear = nn.Linear(num_filters, out_size)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text):
        text = text.permute(1, 0)     # [batch_size, seq_len]
        embedded = self.embedding(text)      # [batch_size, seq_len, emb_dim]
        embedded = embedded.unsqueeze(1)    # [batch_size, 1, seq_len, emb_dim]
        conved = F.relu(self.conv(embedded))   # [batch_size, num_filters, seq_len-filter_size+1]
        conved = conved.squeeze(3)
        pooled = F.max_pool1d(conved, conved.shape[2])   # 把第二个维度压扁， [batch_size, numf, 1]
        pooled = pooled.squeeze(2)    # [batch_size, num_filters]
        pooled = self.dropout(pooled)   # [batch_size, num_filters]
        
        return self.linear(pooled)

CNN网络的强大在于可以使用多种卷积核，进行多层卷积操作，这时候可以把这些卷积层写道一个ModuleList里

In [25]:
class CNN_Model(nn.Module):
    def __init__(self, vocab_size, embedding_dim, n_filters, filter_sizes, output_dim, dropout, pad_idx):
        super(CNN_Model, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=pad_idx)
        self.convs = nn.ModuleList([
            nn.Conv2d(in_channels=1, out_channels=n_filters, kernel_size=(fs, embedding_dim))
            for fs in filter_sizes
        ])
        self.fc = nn.Linear(len(filter_sizes) * n_filters, output_dim)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text):
        text = text.permute(1, 0)   # [batch_size, seq_len]
        embedded = self.embedding(text)  # [batch_size, seq_len, embed_dim]
        embedded = embedded.unsqueeze(1)   # [batch_size, 1, seq_len, embed_dim]
        conved = [F.relu(conv(embedded)).squeeze(3) for conv in self.convs]
        # conv_n: [batch_size, num_filters, seq_len-fliter[n]+1]
        
        pooled = [F.max_pool1d(conv, conv.shape[2]).squeeze(2) for conv in conved]
        # pooled_n: [batch_size, n_filters]
        
        cat = self.dropout(torch.cat(pooled, dim=1))  # [batch_size, n_filters*len(filter_size)]
        return self.fc(cat)

建立模型

In [26]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
N_FILTERS = 100
OUTPUT_DIM = 1
#FILTER_SIZE = 3
FILTER_SIZE = [3, 4, 5]   # 使用多层卷积， 每一层用不同的卷积核
DROPOUT = 0.5
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

model = CNN_Model(INPUT_DIM, EMBEDDING_DIM, N_FILTERS, FILTER_SIZE, OUTPUT_DIM, DROPOUT, PAD_IDX)

In [ ]:
optimizer = optim.Adam(model.parameters())
criterion = nn.BCEWithLogitsLoss()   # 这个是加了sigmoid的损失函数
model = model.to(device)
criterion = criterion.to(device)

N_EPOCHS = 10

best_valid_loss = float('inf')

BATCH_SIZE = 64

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), batch_size=BATCH_SIZE, device=device)

[TEXT.vocab.itos[i] for i in next(iter(train_iterator)).text[1, :]]
for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'CNN-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

In [ ]:
model.load_state_dict(torch.load('CNN-model.pt'))
test_loss, test_acc = evaluate(model, test_iterator, criterion)
print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

# 总结

- PyTorch搭建神经网络，先搭框架，从下往上展开，从forward到init, 再到init的参数
- 神经网络写前向传播的时候，养成核对张量维度的习惯
- 神经网络训练的时候，可以把train和evaluate写成函数的方式，之后换模型也不用改变训练代码
- 保存最好的模型，打印日志信息观察模型是否正常工作

- 关于情感分类时训练模型的技巧，embedding的初始化参数可以用已经训练好的词向量，在这个基础上进行微调
- 情感分类或者语言模型，有个torchtext包可以准备数据，数据初始化或者是构建成Iterator。